Import and sendo db informations about file received



In [8]:
import pyodbc

# Configurações de conexão ao banco de dados com autenticação do Windows
server = 'JNR'
database = 'master'
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Palavras-chave a serem substituídas e suas substituições
replacements = {
    'batata': 'daleste'
}

# Conectar ao banco de dados
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

# Nome da stored procedure a ser alterada
procedure_name = 'proc_test'

try:
    # Obter a definição atual da stored procedure
    cursor.execute(f"SELECT OBJECT_DEFINITION(OBJECT_ID('{procedure_name}')) AS ROUTINE_DEFINITION")
    procedure_definition = cursor.fetchone().ROUTINE_DEFINITION

    if procedure_definition is not None:
        # Localizar o índice da cláusula 'AS' na definição da stored procedure
        index_as = procedure_definition.upper().find('AS')

        if index_as != -1:
            # Obter a parte da definição após a cláusula 'AS'
            new_definition = procedure_definition[index_as + 2:].strip()

            # Substituir as palavras-chave na parte da definição após 'AS'
            for old_word, new_word in replacements.items():
                new_definition = new_definition.replace(old_word, new_word)
            
            # Preparar o comando para alterar a stored procedure
            alter_procedure_command = f"""
            ALTER PROCEDURE [dbo].[{procedure_name}]
            AS
            {new_definition}
            """

            # Atualizar a stored procedure no banco de dados
            cursor.execute(alter_procedure_command)
            print(f"A parte da stored procedure '{procedure_name}' após 'AS' foi atualizada com sucesso.")

            # Confirmar as alterações
            connection.commit()

        else:
            print(f"Não foi possível encontrar a cláusula 'AS' na definição da stored procedure '{procedure_name}'.")

    else:
        print(f"Stored procedure '{procedure_name}' não encontrada.")

except Exception as e:
    print(f"Erro ao tentar alterar a stored procedure '{procedure_name}': {e}")

finally:
    # Fechar a conexão
    cursor.close()
    connection.close()


A parte da stored procedure 'proc_test' após 'AS' foi atualizada com sucesso.
